In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', None)

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from gensim.models import Word2Vec

from pandarallel import pandarallel
pandarallel.initialize()

from preprocessing import preprocess
from tokenizer import tokenize
from utils import read_file, dummy
from pipeline import ThaiPreprocessor, ThaiTokenizer, MeanEmbeddingVectorizer

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [2]:
SEED = 3

# WISESIGHT Sentiment Dataset

In [3]:
pos_msgs = read_file("datasets/wisesight-sentiment/pos.txt")
neu_msgs = read_file("datasets/wisesight-sentiment/neu.txt")
neg_msgs = read_file("datasets/wisesight-sentiment/neg.txt")

In [4]:
ws_df = pd.DataFrame(columns=["text", "sentiment"])

pos_df = pd.DataFrame(data={"text": pos_msgs, "sentiment": "positive"})
neu_df = pd.DataFrame(data={"text": neu_msgs, "sentiment": "neutral"})
neg_df = pd.DataFrame(data={"text": neg_msgs, "sentiment": "negative"})

ws_df = ws_df.append(pos_df, ignore_index=True)
ws_df = ws_df.append(neu_df, ignore_index=True)
ws_df = ws_df.append(neg_df, ignore_index=True)

## Data Cleansing
In order to ignore emoticon and emoji, it has to be preprocessed each message, and remove all of empty string out of our dataset.

In [5]:
%time ws_df['preprocessed_text'] = ws_df['text'].parallel_apply(preprocess)

CPU times: user 117 ms, sys: 86.9 ms, total: 204 ms
Wall time: 50.7 s


In [6]:
mask = ws_df['preprocessed_text'] == ''
ws_df.loc[mask, 'preprocessed_text'] = np.nan

In [7]:
ws_df.dropna(subset=['preprocessed_text'], inplace=True)

In [8]:
ws_df.sample(10)

,text,sentiment,preprocessed_text
9785,MU-X THE ICONIC😊😊😊😊,neutral,mu x the iconic
3211,กูไม่กินเบียร์ กินแค่285 B52 Smirnoff โชจู,positive,กูไม่กินเบียร์ กินแค่ WSNUMBER b WSNUMBER smirnoff โชจู
15585,ความสวยกินกันไม่ลงจริงๆ....เทียบสเป็ค Toyota C-HR และ Honda HR-V 2018 รุ่นท็อปทั้งคู่ อ็อพชั่นใครแน่นกว่า?,neutral,ความสวยกินกันไม่ลงจริงๆ เทียบสเป็ค toyota c hr และ honda hr v WSNUMBER รุ่นท็อปทั้งคู่ อ็อพชั่นใครแน่นกว่า ?
6072,พาแม่ไปซิ,neutral,พาแม่ไปซิ
26087,ม่ายๆละคะ. ขนาดผ้าอนามัยแบบสอดยังใช้ไม่ได้ อันนี้ใหญ่กว่า. จะใส่ยังไง. ใช้เสร็จมีการนำมาใช้อีกไม่เอา หรอก. อีกอย่างสรีระ คนไทยกับต่างชาติไม่เหมือนกันนะ ความยืดหยุ่นเค้าอจมีเ้ยอะกว่า. จะสอดใส่ยังไงก้อได้ แต่คนไทยคงไม่เหมาะ. ใส่แบบผ้าอนามัย แบบเดิมดีอยู่ละคะ. ยังไง4-5ชม.เราก้อ. เราก้อเข้าห้องน้ำอยู่แล้ว. เราเป็นคนแพ้ผ้าอนามัย. ทุกเดือนที่เป็น. จะเป็็นผื่น. แต่พอมีแบบ นุ่มนวลต่อผิว จะเลือกใช้ถึงราคาจะสูงกว่าแต่ พอเมนหายไม่ต้องมารักษาผื่นแพ้อีก.,negative,ม่ายๆละคะ ขนาดผ้าอนามัยแบบสอดยังใช้ไม่ได้ อันนี้ใหญ่กว่า จะใส่ยังไง ใช้เสร็จมีการนำมาใช้อีกไม่เอา หรอก อีกอย่างสรีระ คนไทยกับต่างชาติไม่เหมือนกันนะ ความยืดหยุ่นเค้าอจมีเ้ยอะกว่า จะสอดใส่ยังไงก้อได้ แต่คนไทยคงไม่เหมาะ ใส่แบบผ้าอนามัย แบบเดิมดีอยู่ละคะ ยังไง WSNUMBER WSNUMBER ชม เราก้อ เราก้อเข้าห้องน้ำอยู่แล้ว เราเป็นคนแพ้ผ้าอนามัย ทุกเดือนที่เป็น จะเป็็นผื่น แต่พอมีแบบ นุ่มนวลต่อผิว จะเลือกใช้ถึงราคาจะสูงกว่าแต่ พอเมนหายไม่ต้องมารักษาผื่นแพ้อีก
18985,เอลเดอร์เบอร์รี่ เป็นผลเบอร์รี่ขนาดเล็กเป็นพวงสี่ม่วงเข้ม มีถิ่นกำเนิดในยุโรป อเมริกาเหนือ และเอเชีย ซึ่งผลไม้นี้ประกอบด้วย สารแอนตี้ออกซิแดนท์ที่แตกต่างกันสูงถึง 60 ชนิด ทำให้มีผลในการรักษาสุขภาพของหัวใจและหลอดเลือดหัวใจได้อย่างดี และกระตุ้นระบบภูมิคุ้มกัน และมีสารอาหารที่จำเป็นต่อระบบภูมิคุ้มกันของร่างกาย โดยมีวิตามินซี กรดผลไม้และน้ำมันที่มีประโยชน์ รวมถึงฟลาโวนอยด์ (Flavonoids) และ แอนโธไซยานิน (Anthocyanins) มีฤทธิ์ต้านปฏิกิริยาออกซิเดชั่นที่ดีเลิศ (Powerful antioxidant) เพราะฉะนั้นการเลือกทาน #เอลเดอร์เบอร์รี่ หรืออาหารเสริมที่มีสารสะกัดจากเอลเดอร์เบอร์รี่ ก็จะเป็นผลดีต่อสุขภาพร่างกาย และผิวพรรณจ้า,neutral,เอลเดอร์เบอร์รี่ เป็นผลเบอร์รี่ขนาดเล็กเป็นพวงสี่ม่วงเข้ม มีถิ่นกำเนิดในยุโรป อเมริกาเหนือ และเอเชีย ซึ่งผลไม้นี้ประกอบด้วย สารแอนตี้ออกซิแดนท์ที่แตกต่างกันสูงถึง WSNUMBER ชนิด ทำให้มีผลในการรักษาสุขภาพของหัวใจและหลอดเลือดหัวใจได้อย่างดี และกระตุ้นระบบภูมิคุ้มกัน และมีสารอาหารที่จำเป็นต่อระบบภูมิคุ้มกันของร่างกาย โดยมีวิตามินซี กรดผลไม้และน้ำมันที่มีประโยชน์ รวมถึงฟลาโวนอยด์ flavonoids และ แอนโธไซยานิน anthocyanins มีฤทธิ์ต้านปฏิกิริยาออกซิเดชั่นที่ดีเลิศ powerful antioxidant เพราะฉะนั้นการเลือกทาน เอลเดอร์เบอร์รี่ หรืออาหารเสริมที่มีสารสะกัดจากเอลเดอร์เบอร์รี่ ก็จะเป็นผลดีต่อสุขภาพร่างกาย และผิวพรรณจ้า
19151,TOYOTA มาพร้อมแนวคิด “LIVE ALIVE…Live Ever Better ออกไป….ใช้ชีวิต” กับเทคโนโลยีใหม่ล่าสุดที่ทำให้ทุกชีวิตไปได้ไกลกว่า พาท่องโลกกว้าง ได้อย่างมั่นใจในทุกเส้นทาง พร้อมจัดแสดง 4 เทคโนโลยีใหม่ มาตรฐานรถยนต์โตโยต้า เพื่อตอบสนองในทุกรูปแบบของการใช้ชีวิตที่ไร้ขีดจำกัด เริ่มต้นด้วยระบบไฮบริดเจเนอเรชั่นที่ 4 ที่พัฒนาแบตเตอรี่ใหม่ให้มีประสิทธิภาพสูงขึ้น เพื่อความทนทานและประหยัดน้ำมันยิ่งขึ้น สถาปัตยกรรมโครงสร้างยานยนต์ใหม่ (Toyota New Global Architecture หรือ TNGA) สร้างประสบการณ์การขับขี่ที่เร้าใจกว่าเคย มาตรฐานความปลอดภัยใหม่ระดับโลก (Toyota Safety Sense หรือ TSS) มาตรฐานความปลอดภัยใหม่เพื่อทุกชีวิต และท้ายสุดกับ Toyota T-Connect Telematics เชื่อมต่อทุกเส้นทางอย่างสมบูรณ์แบบ พระเอกคงหนีไม่พ้น All New Toyota CH-R ..เอาเป็นว่าตามไปล้ำกันได้ทุกวันตั้งแต่วันนี้ ถึง 8 เมษายนนี้ที่งานมอเตอร์โชว์ ครั้งที่ 39 เมืองทองธานี,neutral,toyota มาพร้อมแนวคิด live alive live ever better ออกไป ใช้ชีวิต กับเทคโนโลยีใหม่ล่าสุดที่ทำให้ทุกชีวิตไปได้ไกลกว่า พาท่องโลกกว้าง ได้อย่างมั่นใจในทุกเส้นทาง พร้อมจัดแสดง WSNUMBER เทคโนโลยีใหม่ มาตรฐานรถยนต์โตโยต้า เพื่อตอบสนองในทุกรูปแบบของการใช้ชีวิตที่ไร้ขีดจำกัด เริ่มต้นด้วยระบบไฮบริดเจเนอเรชั่นที่ WSNUMBER ที่พัฒนาแบตเตอรี่ใหม่ให้มีประสิทธิภาพสูงขึ้น เพื่อความทนทานและประหยัดน้ำมันยิ่งขึ้น สถาปัตยกรรมโครงสร้างยานยนต์ใหม่ toyota new global architecture หรือ tnga สร้างประสบการณ์การขับขี่ที่เร้าใจกว่าเคย มาตรฐานความปลอดภัยใหม่ระดับโลก toyota safety sense ห

In [9]:
ws_df['sentiment'].value_counts()

neutral     14518
negative     6816
positive     4734
Name: sentiment, dtype: int64

## Splitting Dataset into Training and Test Sets

In [10]:
ws_train, ws_test = train_test_split(ws_df, test_size=0.1, random_state=SEED)

## Word Frequency Counting
The most common and straightforward technique to transform each sequence of words to a feature vector, it is also called CountVectorizer for scikit-learn library.

In [11]:
pipeline_count = make_pipeline(
    ThaiPreprocessor(),
    ThaiTokenizer(),
    CountVectorizer(preprocessor=dummy, tokenizer=dummy),
    LogisticRegression(random_state=SEED, n_jobs=-1)
)

%time pipeline_count.fit(ws_train['text'], ws_train['sentiment'])

CPU times: user 1min 9s, sys: 1.74 s, total: 1min 11s
Wall time: 2min 1s


Pipeline(steps=[('thaipreprocessor', ThaiPreprocessor()),
                ('thaitokenizer', ThaiTokenizer()),
                ('countvectorizer',
                 CountVectorizer(preprocessor=<function dummy at 0x1261a4670>,
                                 tokenizer=<function dummy at 0x1261a4670>)),
                ('logisticregression',
                 LogisticRegression(n_jobs=-1, random_state=3))])

In [12]:
predicted = pipeline_count.predict(ws_test['text'])

In [13]:
print(classification_report(ws_test['sentiment'], predicted, digits=4))

              precision    recall  f1-score   support

    negative     0.7544    0.6321    0.6878       685
     neutral     0.7104    0.8597    0.7779      1418
    positive     0.5931    0.3730    0.4580       504

    accuracy                         0.7058      2607
   macro avg     0.6859    0.6216    0.6412      2607
weighted avg     0.6992    0.7058    0.6924      2607



## Term Frequency-Inverse Document Frequency (TF-IDF)
Typically, TF-IDF is one of well-known feature transformation techniques. There are different points between word frequency and TF-IDF techniques, TF-IDF can capture dominant words for each document in a dataset. Not just counting the word occurrence, TF-IDF normalizes word counts in a focus document with others in the same dataset. However, these techniques are limited to the certain dataset and also fall into an Out of Vocabulary (OOV) problem.

In [14]:
pipeline_tfidf = make_pipeline(
    ThaiPreprocessor(),
    ThaiTokenizer(),
    TfidfVectorizer(preprocessor=dummy, tokenizer=dummy),
    LogisticRegression(random_state=SEED, n_jobs=-1)
)
%time pipeline_tfidf.fit(ws_train['text'], ws_train['sentiment'])

CPU times: user 1min 30s, sys: 2.26 s, total: 1min 32s
Wall time: 2min 48s


Pipeline(steps=[('thaipreprocessor', ThaiPreprocessor()),
                ('thaitokenizer', ThaiTokenizer()),
                ('tfidfvectorizer',
                 TfidfVectorizer(preprocessor=<function dummy at 0x1261a4670>,
                                 tokenizer=<function dummy at 0x1261a4670>)),
                ('logisticregression',
                 LogisticRegression(n_jobs=-1, random_state=3))])

In [15]:
predicted = pipeline_tfidf.predict(ws_test['text'])

In [16]:
print(classification_report(ws_test['sentiment'], predicted, digits=4))

              precision    recall  f1-score   support

    negative     0.7556    0.6409    0.6935       685
     neutral     0.7036    0.8822    0.7829      1418
    positive     0.6774    0.3333    0.4468       504

    accuracy                         0.7127      2607
   macro avg     0.7122    0.6188    0.6411      2607
weighted avg     0.7122    0.7127    0.6944      2607



## Word Embedding as Vectorizer
According the OOV problem, word embedding is one of techniques to address the problem. Word Embedding is a Language Model (LM), It is trained from a large corpus, it encode each word into vector, encoded vectors represent words. The embedding model can be applied to predict next word from a given context and also captures semantic between words. In order to use the embedding model as a vectorizer, a given sentence are encoded to each vector and average them to represent the sentence.

In [17]:
# load pre-trained word embedding model
w2v_model = Word2Vec.load("models/tweet_embedding_256.model")

In [18]:
pipeline_w2v = make_pipeline(
    ThaiPreprocessor(),
    ThaiTokenizer(),
    MeanEmbeddingVectorizer(w2v_model=w2v_model),
    LogisticRegression(random_state=SEED, n_jobs=-1)
)
%time pipeline_w2v.fit(ws_train['text'], ws_train['sentiment'])

CPU times: user 1min 25s, sys: 2.1 s, total: 1min 27s
Wall time: 2min 48s


Pipeline(steps=[('thaipreprocessor', ThaiPreprocessor()),
                ('thaitokenizer', ThaiTokenizer()),
                ('meanembeddingvectorizer',
                 <pipeline.MeanEmbeddingVectorizer object at 0x12d7778e0>),
                ('logisticregression',
                 LogisticRegression(n_jobs=-1, random_state=3))])

In [19]:
predicted = pipeline_w2v.predict(ws_test['text'])

In [20]:
print(classification_report(ws_test['sentiment'], predicted, digits=4))

              precision    recall  f1-score   support

    negative     0.6161    0.5153    0.5612       685
     neutral     0.6421    0.8188    0.7198      1418
    positive     0.6062    0.2718    0.3753       504

    accuracy                         0.6333      2607
   macro avg     0.6215    0.5353    0.5521      2607
weighted avg     0.6283    0.6333    0.6115      2607



## Predicting Messages

In [21]:
test_df = ws_test[['text', 'sentiment']]

test_df['count_vectorizer'] = test_df['text'].apply(lambda text: pipeline_count.predict(text)[0])
test_df['tfidf_vectorizer'] = test_df['text'].apply(lambda text: pipeline_tfidf.predict(text)[0])
test_df['mean_embedding_vectorizer'] = test_df['text'].apply(lambda text: pipeline_w2v.predict(text)[0])

In [22]:
ws_test.sample(5)

,text,sentiment,preprocessed_text
20008,เมกาบางนา ไม่ร่วมเด้อ,negative,เมกาบางนา ไม่ร่วมเด้อ
25748,ทีมขุนเรืองฟังทางนี้!! ถ้าอยากเป็นผู้โชคดี ไปใกล้ชิดกับ ปั้นจั่น ปรมะ ในกิจกรรม Cool Day Fine Way กับ สุดสัปดาห์ และ มิตซูบิชิ แอททราจ ไปที่ Facebook สุดสัปดาห์แฟนคลับ หรือ Mitsubishi Motors Thailand ด่วนๆ พลาดแล้วจะเสียใจนะบอกเลย!!,negative,ทีมขุนเรืองฟังทางนี้ ! ! ถ้าอยากเป็นผู้โชคดี ไปใกล้ชิดกับ ปั้นจั่น ปรมะ ในกิจกรรม cool day fine way กับ สุดสัปดาห์ และ มิตซูบิชิ แอททราจ ไปที่ facebook สุดสัปดาห์แฟนคลับ หรือ mitsubishi motors thailand ด่วนๆ พลาดแล้วจะเสียใจนะบอกเลย ! !
21156,มันเกี่ยวกับภาษีที่จะหดหายล้วนๆ,negative,มันเกี่ยวกับภาษีที่จะหดหายล้วนๆ
7721,มันอันตรายนะจ่า,neutral,มันอันตรายนะจ่า
22693,บาบีกอนบอกไม่ใช่พยาธิ แต่เป็นอีสิ่งนี้ น่าสะพรึงกลัวพอๆกัน,negative,บาบีกอนบอกไม่ใช่พยาธิ แต่เป็นอีสิ่งนี้ น่าสะพรึงกลัวพอๆกัน
